In [5]:
import matplotlib.pylab as plt
import numpy as np

prev_epoch = 0;
maxsamp = 0;
loss = {};
with open("./log/ukiyoe2photo/loss.log") as file:
    for line in file:
        sline = line.rstrip().split(',');
        if (len(sline) == 7):
            prev_epoch = int(sline[0]);
            if (int(sline[1]) > maxsamp):
                maxsamp = int(sline[1]);
            loss[(int(sline[0]),int(sline[1]))] = list(map(float, sline[-5:]));
        elif (len(sline) > 7):
            if (int(sline[-6]) > maxsamp):
                maxsamp = int(sline[-6]);
            loss[(int(prev_epoch+1),int(sline[-6]))] = list(map(float, sline[-5:]));

print(maxsamp);
lossmeans = {}
for i in range(0,200):
    sumnum = [0,0,0,0,0];
    count = 1;
    for j in range(0,maxsamp):
        if ((i,j) in loss):
            sumnum = np.add(sumnum,loss[(i,j)]);
            count = count+1;
    print(i);
    if (count != 1):
        print(np.divide(sumnum, (count-1)));
    else:
        print("No epoch!")
    
    
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        
        

6286
0
[8.41068568 2.40214446 0.36815388 5.09509036 0.37034764]
1
[7.11729525 1.98979925 0.32130718 4.19591506 0.32890173]
2
[6.59347572 1.80535536 0.29495691 3.83178282 0.30085246]
3
[6.19685073 1.66586811 0.28282182 3.55815802 0.29087344]
4
[5.95946086 1.5756515  0.26561488 3.3977655  0.27417229]
5
[5.81295984 1.51560606 0.26521595 3.29404932 0.2716108 ]
6
[5.63666435 1.45905914 0.26704956 3.16990432 0.27182421]
7
[5.5394866  1.40374009 0.25574034 3.11167004 0.26105931]
8
[5.4637414  1.36899661 0.24767885 3.0425251  0.25372482]
9
[5.3754522  1.33353017 0.24394542 2.98385014 0.24845275]
10
[5.29477429 1.29769425 0.24218283 2.93414279 0.24455236]
11
[5.240522   1.2757036  0.2378908  2.89254053 0.24009362]
12
[5.18551626 1.24317047 0.23101053 2.84840148 0.23617025]
13
[5.15756092 1.22906441 0.21945065 2.81528485 0.22592643]
14
[5.09739728 1.20087154 0.22051998 2.77510067 0.22512645]
15
[5.04101852 1.17958382 0.21521027 2.73202319 0.22123088]
16
[5.01057429 1.15677653 0.20995182 2.707830

147
[3.9135847  0.61689732 0.11569226 1.74078564 0.12227231]
148
[3.90989924 0.61037363 0.1151168  1.73351016 0.11933676]
149
[3.92411883 0.61490832 0.11655845 1.73905418 0.1190774 ]
150
[3.92776426 0.61553083 0.11502254 1.74864321 0.12024821]
151
[3.89986841 0.60989431 0.11545942 1.72915138 0.12120048]
152
[3.90322476 0.60717029 0.11453498 1.72759395 0.11862933]
153
[3.89486944 0.60602384 0.11457797 1.73024601 0.11942897]
154
[3.89282437 0.60384065 0.11314903 1.72196413 0.11812905]
155
[3.89402405 0.60299514 0.11356793 1.71934728 0.11887999]
156
[3.892677   0.6006073  0.11241211 1.7163721  0.11766084]
157
[3.88370539 0.59978453 0.11341377 1.71530007 0.1179722 ]
158
[3.88441618 0.60071    0.11337037 1.71383953 0.11872925]
159
[3.88644606 0.59863819 0.11653138 1.71510057 0.12019887]
160
[3.8873188  0.59895508 0.11421039 1.71621979 0.11946064]
161
[3.87226726 0.59726525 0.11446111 1.70783089 0.11943356]
162
[3.87552882 0.59746098 0.11060944 1.7010882  0.11771591]
163
[3.86514506 0.594013

['17.716349', '5.711968', '0.888718', '11.762389']
